In [1]:


import torch 
from torch.utils.data import Dataset, DataLoader
import captum
from captum.attr import IntegratedGradients
from captum.attr import visualization
import pandas as pd
import os
import pathlib
import torch
import pandas as pd
import transformers
from underthesea import word_tokenize
from torch.utils.data import Dataset, DataLoader
import numpy as np 

In [2]:
import torch
import torch.nn as nn
from transformers import AutoModel,AutoTokenizer
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
class DynamicClassifier(nn.Module):
    def __init__(self, 
                input_dim=768, 
                num_classes=4, 
                dropout_rate=0.25, 
                num_layers=3):
        super().__init__()
        layers = []
        for i in range(num_layers):
            output_dim = input_dim // 2
            layers.append(nn.Linear(input_dim, output_dim))
            layers.append(nn.ReLU())
            if i < num_layers - 1:  # No dropout after last layer before classification
                layers.append(nn.Dropout(dropout_rate))
            input_dim = output_dim
        layers.append(nn.Linear(input_dim, num_classes))
        layers.append(nn.Softmax(dim=1))
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, feature):
        return self.model(feature)

class ViTBERTClassifier(nn.Module):
    def __init__(self, 
                pretrained_model_name, 
                num_classes=4, 
                dropout_rate=0.25, 
                num_layers=4):
        super().__init__()
        self.bert = AutoModel.from_pretrained(pretrained_model_name)
        self.classifier = DynamicClassifier(input_dim=self.bert.config.hidden_size,
                                            num_classes=num_classes,
                                            dropout_rate=dropout_rate,
                                            num_layers=num_layers)
    
    def forward(self, input_ids, attention_mask=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state[:, 0, :]
        logits = self.classifier(last_hidden_state)
        return logits
    def compute_l1_loss(self, w):
        return torch.abs(w).sum()


In [3]:

model= torch.load("/home/khanhhiep/Code/Khanh/Khiem/MyBachelorThesis/notebooks/best_model/best_model.pt",weights_only= False, map_location='cpu')

model.eval()
device = "cuda"

2025-06-12 16:08:14.518141: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749712094.536882 1439086 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749712094.542608 1439086 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749712094.556290 1439086 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749712094.556319 1439086 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749712094.556321 1439086 computation_placer.cc:177] computation placer alr

In [4]:
class ViTBERT(Dataset):
    def __init__(self, data_path, tokenizer=None):
        self.dataset = pd.read_csv(data_path)
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(tokenizer)

    def __getitem__(self, idx):
        text = self.dataset.iloc[idx, 0]  # Extract text from dataframe
        label = self.dataset.iloc[idx, 1]-1.0  # Extract label

        # Tokenization
        tokenized_text = self.tokenizer(text, add_special_tokens=True, max_length=100, padding='max_length', truncation=True, return_tensors='pt')

        # Extract fields from tokenized text
        input_ids = tokenized_text['input_ids'].squeeze().long()
        attention_mask = tokenized_text['attention_mask'].squeeze().long()

        # Convert label to tensor
        label = torch.tensor(label, dtype=torch.long)  # Changed dtype to torch.long

        # Return tensors
        return tokenized_text ,input_ids, attention_mask, label, text  # Return attention_mask

    def __len__(self):
        return len(self.dataset)

In [5]:
# dataset = ViTBERT(data_path="/media/data3/home/khiemdd/ViTBERT/dataset/dataset_chi_ha_hieu/dataset_final_train_after.csv", 
#                   tokenizer="demdecuong/vihealthbert-base-word")
# dataloader = DataLoader(dataset,batch_size= 1)
# tokenized_text,input_ids, attention_mask, labels, text = next(iter(dataloader))

In [6]:
# print("Tokenized Text:", tokenized_text)

# print("\nInput IDs:", input_ids)
# print("Shape of Input IDs:", input_ids.shape)
# print("Type of Input IDs:", type(input_ids))

# print("\nAttention Mask:", attention_mask)
# print("Shape of Attention Mask:", attention_mask.shape)
# print("Type of Attention Mask:", type(attention_mask))

# print("\nLabels:", labels)
# print("Shape of Labels:", labels.shape)

# print("\nText:", text)
# print("Shape of Text:", len(text))


In [7]:
import torch.nn.functional as F

In [8]:
target_classes = ["rất nhẹ", "nhẹ", "vừa", "nặng"]
def predict(X_batch):
    preds = model(X_batch)
    return F.softmax(preds, dim=-1)

In [9]:
# print(input_ids.shape)
# print(labels.shape)

In [10]:
# input_ids.unsqueeze(0).shape

In [11]:
def print_gpu_memory():
    print(f'Allocated: {torch.cuda.memory_allocated()}')
    print(f'Cached: {torch.cuda.memory_reserved()}')

# Clear cache and print memory usage
torch.cuda.empty_cache()
print_gpu_memory()

Allocated: 0
Cached: 0


In [12]:
model = torch.load("/home/khanhhiep/Code/Khanh/Khiem/MyBachelorThesis/notebooks/best_model/best_model@.pt",weights_only= False, map_location='cpu')

In [13]:
model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)

In [14]:
text = "chấn_thương sọ não máu tụ dưới màng_cứng vùng thái_dương đỉnh phải dập não xuất_huyết bao trong phải vết_thương phần_mềm vùng thái_dương đỉnh phải gãy kín đầu dưới 2 xương cẳng_chân trái gãy kín chỏm xương đốt bàn ngón i ii 3 4 chân trái do tai_nạn giao_thông tăng huyết_áp đái_tháo_đường"


model_name = "medicalai/ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer_text = tokenizer(text, add_special_tokens=True, max_length=512, padding='max_length', truncation=True, return_tensors='pt')
print(tokenizer_text)

{'input_ids': tensor([[   101,  62628,    168,  23886,  13891,  10380,  10132,  10133,  43024,
          13055,  23000,  10116,  23912,    168,  28930,  10985,  10240, 101379,
            168,  23000,  10376,  16895,  16317,  99142,  11542,  10143,  10410,
          10132,  10133,    192,  48439,    168,  33901,  10308,  18323,  10504,
          99142,  11542,  56975,    168,  23886,  13891,  99142,  10206,    168,
          10911,  10147,  10985,  10240, 101379,    168,  23000,  10376,  16895,
          16317,  99142,  11542,  29435,  37403, 110175,  23000,  10116,    123,
            192,  21187,  10376,  10944,  10240,    168,  62628,  34101,  29435,
          37403,  98696,    192,  21187,  10376, 105829,  10123,  11915,  10743,
          10263,    177,  59139,    124,    125,  62628,  34101,  10149,  12815,
            168,  10799,  25216,    168,  77586,  13891,  15176,  10240,  33901,
          10308,    168,  26219,  51410,    168,  89499,    168,    299,  21187,
          1037

In [15]:
model.to(device)
input_ids = tokenizer_text["input_ids"]
attention_mask = tokenizer_text["attention_mask"]

In [16]:
input_ids = input_ids.to(torch.device("cuda:0"))
attention_mask = attention_mask.to(torch.device("cuda"))
model(input_ids, attention_mask).logits


tensor([[-1.6579, -0.9015,  0.0828,  2.5161]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [17]:
def predict(inputs, token_type_ids=None, position_ids=None, attention_mask=None):
    output = model(inputs, token_type_ids=token_type_ids,
                 position_ids=position_ids, attention_mask=attention_mask, )
    return output.start_logits, output.end_logits

In [18]:
import torch
import captum
from captum.attr import IntegratedGradients
from transformers import RobertaTokenizer


def preprocess(text):
    # Tokenize the input text and get input IDs and attention mask
    encoding = tokenizer(text, add_special_tokens=True, max_length=512, padding='max_length', truncation=True, return_tensors='pt')
    return encoding['input_ids'], encoding['attention_mask']



In [19]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)

In [20]:
# def forward_func(input_ids, attention_mask):
#     # Get the output of the model
#     input_ids = input_ids.to(torch.device("cuda:0"))
#     attention_mask = attention_mask.to(torch.device("cuda:0"))
#     input_ids = torch.tensor(input_ids,dtype= "float",requires_grad=True)
#     attention_mask = torch.tensor(attention_mask,dtype="float",requires_grad=True)
#     outputs = model(input_ids=input_ids, attention_mask=attention_mask)
#     # Get the classification logits
#     logits = outputs
#     return logits
def forward_func(embeddings, attention_mask):
    attention_mask = attention_mask.to(embeddings.device)
    outputs = model(inputs_embeds=embeddings, attention_mask=attention_mask)
    return outputs.logits


def explain(model, text, target_class):
    input_ids, attention_mask = preprocess(text)
    input_ids = input_ids.to(model.device)
    attention_mask = attention_mask.to(model.device)

    # Lấy embedding đầu vào
    embedding_layer = model.get_input_embeddings()
    embeddings = embedding_layer(input_ids)
    embeddings.requires_grad_()

    # Tính attribution
    ig = IntegratedGradients(forward_func)
    attributions, delta = ig.attribute(
        inputs=embeddings,
        additional_forward_args=(attention_mask,),
        target=target_class,
        return_convergence_delta=True
    )

    tokens = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())
    return attributions, tokens, delta


In [21]:
print("Number of labels:", model.config.num_labels)


Number of labels: 4


In [22]:
import numpy as np

text = "traumatic_brain injury_trademark_compensation_soft left summit due to fall"
target_class = 1

# Run explanation
attributions, tokens, delta = explain(model, text, target_class)

# Display results (mean of attribution vector per token)
for token, attribution in zip(tokens, attributions[0].tolist()):
    score = np.mean(attribution)
    print(f"{token:>12}: {score:.4f}")


       [CLS]: 0.0000
      trauma: 0.0002
       ##tic: -0.0000
           _: 0.0001
       brain: -0.0003
      injury: -0.0005
           _: 0.0002
       trade: -0.0008
      ##mark: -0.0000
           _: 0.0001
compensation: -0.0007
           _: 0.0002
        soft: 0.0004
        left: 0.0005
      summit: -0.0003
         due: -0.0001
          to: -0.0001
        fall: -0.0001
       [SEP]: -0.0006
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
       [PAD]: 0.0000
  

In [23]:
from IPython.display import display, HTML
import numpy as np

def visualize_token_attributions(tokens, attributions, normalize=True):
    html_output = ""

    # Compute 1D scores per token (mean across embedding dim)
    scores = [np.mean(attr) for attr in attributions]

    # Optional: normalize scores to [0, 1] for coloring
    if normalize:
        max_score = max(abs(np.min(scores)), abs(np.max(scores))) + 1e-10
        scores = [score / max_score for score in scores]

    for token, score in zip(tokens, scores):
        red = f"rgba(255, 0, 0, {score})" if score > 0 else f"rgba(0, 0, 255, {abs(score)})"
        html_output += f"<span style='background-color: {red}; padding:2px; margin:1px; border-radius:3px'>{token}</span> "

    display(HTML(html_output))


In [27]:
from captum.attr import visualization
import torch
import numpy as np

def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records, tokenizer):
    # Reduce attribution vector (embedding dim) -> 1D per token
    attributions_sum = attributions.sum(dim=2).squeeze(0)  # shape: (seq_len,)
    attributions_norm = attributions_sum / torch.norm(attributions_sum)
    attributions_np = attributions_norm.cpu().detach().numpy()

    # Tokenize again to match attribution positions
    input_ids = tokenizer(text, return_tensors='pt', truncation=True)['input_ids'][0]
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Ensure same length
    if len(attributions_np) > len(tokens):
        attributions_np = attributions_np[:len(tokens)]
    elif len(tokens) > len(attributions_np):
        tokens = tokens[:len(attributions_np)]

    # Add visualization record
    vis_data_records.append(visualization.VisualizationDataRecord(
        attributions_np,
        pred,
        str(pred_ind),
        str(label),
        str(pred_ind),  # e.g., predicted class description
        attributions_np.sum(),  # total attribution score
        tokens,
        delta
    ))
from captum.attr import visualization

# Your input
text = "Subperitoneal hematoma chronic secondary hemorrhage due 4th week surgery_conductive irrigation_closed hematomas on both sides hemorrhage_first day hemorrhage_urinary obstruction_disconnection_sugar"
target_class = 1
pred = 0.95  # mock prediction confidence
pred_ind = 1  # predicted class index
label = 1     # ground truth label (or something else)
delta = 0.01  # convergence delta returned by IG

# Compute attribution
attributions, tokens, delta = explain(model, text, target_class)

# Visualization list
vis_data_records = []

# Add to Captum visualizer
add_attributions_to_visualizer(
    attributions,
    text,
    pred,
    pred_ind,
    label,
    delta,
    vis_data_records,
    tokenizer
)

# Display (inside Jupyter)
visualization.visualize_text(vis_data_records)


In [28]:
from IPython.display import HTML, display
import numpy as np

def average_attribution_for_words(tokens, attributions):
    words = []
    word_attributions = []
    current_word = ""
    current_attr = []

    for token, attr in zip(tokens, attributions):
        if token == "[PAD]":
            if "[PAD]" not in words:  # Chỉ thêm 1 lần
                words.append("[PAD]")
                word_attributions.append(0.0)
            continue  # Bỏ qua các PAD tiếp theo

        if token.startswith("##"):
            current_word += token[2:]
            current_attr.append(np.mean(attr))
        else:
            if current_word:
                words.append(current_word)
                word_attributions.append(np.mean(current_attr))
            current_word = token
            current_attr = [np.mean(attr)]

    if current_word:
        words.append(current_word)
        word_attributions.append(np.mean(current_attr))

    return words, word_attributions

def visualize_text_attributions(tokens, attributions):
    words, word_attrs = average_attribution_for_words(tokens, attributions)

    max_score = max(abs(np.min(word_attrs)), abs(np.max(word_attrs))) + 1e-8
    html = "<div style='font-family:monospace; line-height:1.8em'>"

    for word, score in zip(words, word_attrs):
        normalized = score / max_score
        r, g, b, alpha = (255, 0, 0, abs(normalized)) if normalized > 0 else (0, 0, 255, abs(normalized))
        html += (
            f"<span style='background-color:rgba({r},{g},{b},{alpha});"
            f"padding:3px; margin:2px; border-radius:4px; display:inline-block;'>"
            f"{word}</span> "
        )

    html += "</div>"
    display(HTML(html))

visualize_text_attributions(tokens, attributions[0].tolist())
